In [1]:
import MetaTrader5 as mt5
import pandas as pd

In [3]:
# initialize and login to MetaTrader5
mt5.initialize()

# Click here to open your own trading account at Tickmill 
# https://www.tickmill.com/eu
login = 25131017
password = 'vT8.iqYHWia!'
server = 'TickmillEU-Demo'

mt5.login(login, password, server)

True

In [23]:
# market order

symbol = 'EURUSD'
volume = 0.1
action = mt5.TRADE_ACTION_DEAL
order_type = mt5.ORDER_TYPE_BUY

stop_loss = 1.0  # set to 0.0 if you don't want SL
take_profit = 1.2  # set to 0.0 if you don't want TP

def get_market_price(symbol, type):
    if type == mt5.ORDER_TYPE_BUY:
        return mt5.symbol_info(symbol).ask
    elif type == mt5.ORDER_TYPE_SELL:
        return mt5.symbol_info(symbol).bid

request = {
    "action": action,
    "symbol": 'EURUSD',
    "volume": 0.1,  # float
    "type": mt5.ORDER_TYPE_BUY,
    "price": get_market_price('EURUSD', 0),
    "sl": stop_loss,  # float
    "tp": take_profit,  # float
    "deviation": 20,
    "magic": 0,
    "comment": "python market order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

OrderSendResult(retcode=10009, deal=403247691, order=549236243, volume=0.1, price=1.0963, bid=1.0963, ask=1.0963, comment='Request executed', request_id=3051753336, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSD', volume=0.1, price=1.0963, stoplimit=0.0, sl=1.0, tp=1.2, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))

In [40]:
# modify SL/TP

stop_loss = 1.05  # set to 0.0 if you don't want SL
take_profit = 1.15  # set to 0.0 if you don't want TP

positions = mt5.positions_get()
print('open positions', positions)

# Working with 1st position in the list and closing it
pos1 = positions[0]

request = {
    'action': mt5.TRADE_ACTION_SLTP,
    'position': pos1.ticket,
    'sl': stop_loss,
    'tp': take_profit
}

res = mt5.order_send(request)
res

open positions (TradePosition(ticket=549258579, time=1703090156, time_msc=1703090156758, time_update=1703090156, time_update_msc=1703090156758, type=0, magic=0, identifier=549258579, reason=0, volume=0.5, price_open=1.09641, sl=0.0, tp=0.0, price_current=1.09649, swap=0.0, profit=4.0, symbol='EURUSD', comment='', external_id=''),)


OrderSendResult(retcode=10009, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=3051753347, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=1.05, tp=1.15, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=549258579, position_by=0))

In [10]:
# close position

positions = mt5.positions_get()
print('open positions', positions)

# Working with 1st position in the list and closing it
pos1 = positions[0]

def reverse_type(type):
    # to close a buy positions, you must perform a sell position and vice versa
    if type == mt5.ORDER_TYPE_BUY:
        return mt5.ORDER_TYPE_SELL
    elif type == mt5.ORDER_TYPE_SELL:
        return mt5.ORDER_TYPE_BUY


def get_close_price(symbol, type):
    if type == mt5.ORDER_TYPE_BUY:
        return mt5.symbol_info(symbol).bid
    elif type == mt5.ORDER_TYPE_SELL:
        return mt5.symbol_info(symbol).ask

request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "position": pos1.ticket,
    "symbol": pos1.symbol,
    "volume": pos1.volume,
    "type": reverse_type(pos1.type),
    "price":get_close_price(pos1.symbol, pos1.type),
    "deviation": 20,
    "magic": 0,
    "comment": "python close order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

open positions (TradePosition(ticket=549223046, time=1703088728, time_msc=1703088728288, time_update=1703088728, time_update_msc=1703088728288, type=0, magic=0, identifier=549223046, reason=3, volume=0.1, price_open=1.0965, sl=0.0, tp=0.0, price_current=1.09676, swap=0.0, profit=2.6, symbol='EURUSD', comment='python market or', external_id=''),)


OrderSendResult(retcode=10009, deal=403242591, order=549228194, volume=0.1, price=1.09676, bid=1.09676, ask=1.09676, comment='Request executed', request_id=3051753323, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSD', volume=0.1, price=1.09676, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python close order', position=549223046, position_by=0))

In [24]:
# partial close position
perc_close = 0.5

positions = mt5.positions_get()
print('open positions', positions)

# Working with 1st position in the list and closing it
pos1 = positions[0]

def reverse_type(type):
    # to close a buy positions, you must perform a sell position and vice versa
    if type == 0:
        return 1
    elif type == 1:
        return 0


def get_close_price(symbol, type):
    if type == 0:
        return mt5.symbol_info(symbol).bid
    elif type == 1:
        return mt5.symbol_info(symbol).ask

request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "position": pos1.ticket,
    "symbol": pos1.symbol,
    "volume": round(pos1.volume * perc_close, 2),  # closing only half
    "type": reverse_type(pos1.type),
    "price":get_close_price(pos1.symbol, pos1.type),
    "deviation": 20,
    "magic": 0,
    "comment": "python partial close order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

open positions ()


IndexError: tuple index out of range

In [25]:
# limit orders

symbol = 'EURUSD'
volume = 0.1

action = mt5.TRADE_ACTION_PENDING
order_type = mt5.ORDER_TYPE_BUY_LIMIT
limit_price = 1.0

stop_loss = 0.9  # set to 0.0 if you don't want SL
take_profit = 1.2  # set to 0.0 if you don't want TP

request = {
    "action": action,
    "symbol": 'EURUSD',
    "volume": 0.1,  # float
    "type": order_type,
    "price": limit_price,
    "sl": stop_loss,  # float
    "tp": take_profit,  # float
    "deviation": 20,
    "magic": 0,
    "comment": "python market order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

OrderSendResult(retcode=10009, deal=0, order=549245467, volume=0.1, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=3051753338, retcode_external=0, request=TradeRequest(action=5, magic=0, order=0, symbol='EURUSD', volume=0.1, price=1.0, stoplimit=0.0, sl=0.9, tp=1.2, deviation=20, type=2, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))

In [33]:
# stop orders

symbol = 'EURUSD'
volume = 0.1

action = mt5.TRADE_ACTION_PENDING
order_type = mt5.ORDER_TYPE_BUY_STOP
stop_price = 1.2

stop_loss = 1.0  # set to 0.0 if you don't want SL
take_profit = 1.3  # set to 0.0 if you don't want TP

request = {
    "action": action,
    "symbol": 'EURUSD',
    "volume": 0.1,  # float
    "type": order_type,
    "price": stop_price,
    "sl": stop_loss,  # float
    "tp": take_profit,  # float
    "deviation": 20,
    "magic": 0,
    "comment": "python market order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

OrderSendResult(retcode=10009, deal=0, order=549255173, volume=0.1, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=3051753344, retcode_external=0, request=TradeRequest(action=5, magic=0, order=0, symbol='EURUSD', volume=0.1, price=1.2, stoplimit=0.0, sl=1.0, tp=1.3, deviation=20, type=4, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))

In [34]:
# Remove Pending Order
pending_orders = mt5.orders_get()
print('open pending orders', pending_orders)

order1 = pending_orders[0]

request = {
    'action': mt5.TRADE_ACTION_REMOVE,
    'order': order1.ticket
}

mt5.order_send(request)

open pending orders (TradeOrder(ticket=549255173, time_setup=1703090046, time_setup_msc=1703090046172, time_done=0, time_done_msc=0, time_expiration=0, type=4, type_time=0, type_filling=2, state=1, magic=0, position_id=0, position_by_id=0, reason=3, volume_initial=0.1, volume_current=0.1, price_open=1.2, sl=1.0, tp=1.3, price_current=1.09654, price_stoplimit=0.0, symbol='EURUSD', comment='python market or', external_id=''),)


OrderSendResult(retcode=10009, deal=0, order=549255173, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=3051753345, retcode_external=0, request=TradeRequest(action=8, magic=0, order=549255173, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))